## PFFP Code Intro

The purpose of this notebook is to detail the features and steps to process PFFP binary code information.

Date: 09/09/2024

Name: Jonathan Moore

## Library/Module importing

This step is so that the libraries required can be accessed by the local code

In [ ]:
import matplotlib.pyplot as plt
import sys
import os
import numpy as np

#### Add the BlueDrop Analysis library to the current path
This is necessary for the time being since the library hasn't been converted a python package yet

In [ ]:
sys.path.append("../../BlueDrop_Analysis_Lib")

### Local Imports
These are imports from the actual BlueDrop Analysis Lib that our team wrote

In [ ]:
# Local loads

# This class represent the Folder that contains the pffp data
from lib.data_classes.pffpFolder import pffpDataFolder

### Set the necessary paths
These paths will be used to retrieve the data, dimensions of the BlueDrop, and Calibration Data


In [ ]:
# Data folder
# folder_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\stark_drops"
folder_dir = os.path.join(os.pardir, "Mouth_1")

# PFFP sensor data
calibration_dir = os.path.join(os.pardir, "calibration_factors/BlueDrop_Calibration_Factors.xlsx")

# PFFP tip information
tip_dir = os.path.join(os.pardir, "calibration_factors/BluedDrop_Cone_Values.xlsx")


### Creating the pffp Data folder

The purpose of the next cell is to create the ```pffp_data_folder``` and load the correct calibration factors to convert the voltages measured by the BlueDrop sensors to engineering units (g's, meters, pressure (kPa))

In [ ]:
# Set the name for the survey 
name = "VIM_Mouth_1"

# Set the id for the pffp
id = 3

# Create the object that will represent the folder with the pffp data in it
pffp_data_folder = pffpDataFolder(folder_dir, pffp_id = id, calibration_factor_dir=calibration_dir, survey_name = name)

# Read the calibration data
pffp_data_folder.read_calibration_excel_sheet()

# Select the specific calibration params
if id == 3:
    pffp_data_folder.get_sensor_calibration_params("Sequim_2021")
elif id == 1:
    pffp_data_folder.get_sensor_calibration_params("March_2023")
elif id ==9:
    pffp_data_folder.get_sensor_calibration_params("October_2023")
else:
    raise ValueError("not a valid pffp id")


#### Display the selected calibration factors

In [ ]:
display(pffp_data_folder.calibration_excel_sheet)

### Find all the pffp files

The next cell finds all the pffp files and stores a reference to them. The way it does this is by searching to find all the ```.bin``` files in the folder. As such, make sure there are no other files in the folder that have a ```.bin``` file extension.

In [ ]:
# Init a pffpFile instances for each binary file and store the results in the folder
pffp_data_folder.store_pffp_files(recursive = False)

# Print Meta-data about the pffp_data_folder including the number of .bin files found.
print(pffp_data_folder) # Information about the folder

### Analyze the files

The purpose of this step is to determine which files have drops in them. 

The possible scenarios are:

#### Drop in file
If the code unequivocally finds all the drops in the file, then the reference to the file object is stored in the ```pffp_drop_files``` list. 

#### No Drop in file
If the code unequivocally finds no drops in the file, then the reference to the file object is stored in the ```pffp_no_drop_files``` list. Additionally, the actual ```.bin``` file is moved into a subdirectory of the current working folder, that will contain the files that don't have drops in them. The default name for this folder is ```no_drop_folder```

#### Code can't figure out if there's a drop or not
When the code can't determine if the file does or does not contain a drop, the files are treated as "funky" files. Similar to the "no drop" files the funky files have their actual '''.bin''' file moved into a subdirectory of the current working folder. The name for this folder is ```funky_folder```. 

#### Accessing the lists that contain the different file types
Given that the folder object in this example is called ```pffp_data_folder```, the different lists can be accessed in the following ways:

* Files that unequivocally contain drops       : ```pffp_data_folder.pffp_drop_files```
* Files that unequivocallyt don't contain drops: ```pffp_data_folder.pffp_no_drop_files```
* Files that have a funky status               : ```pffp_data_folder.pffp_funky_files```


In [ ]:
# Figures out if the .bin file has a drop
pffp_data_folder.analyze_all_files(store_df=True, use_pore_pressure=True, 
                              select_accel=["2g_accel", "18g_accel", "50g_accel", "250g_accel"],
                              debug = False)

### Processing Drop Files

The next cell processes the files that unequivocally have drops in them.

This means that for each drop in each file that contains a drop the following are done:

* An object is created to represent each drop 
* The start and end of the drop are found
* The acceleration data is integrated to get the velocity and displacement

In [ ]:
# Figure out the beginning and the end of the drops if the files contain drops
pffp_data_folder.process_drop_files()

### Processing Funky Files

For funky files there's at least one drop in the file that the code isn't sure if it's a drop or not. As such, those files need to be processed by hand. The following cell details the steps that are necessary to do that.

If the code doesn't find any funky files that this step isn't necessary.

In [ ]:
# Steps to process funky files

# Loop over all the files that are funky
for file in pffp_data_folder.pffp_funky_files:

    # Manual Process the drop. This method will take you through the process of manually processing the drop
    file.manually_process_drops(interactive_plot=True)

### Print out folder metadata

Printing out the folder meta-data again to see how it's been updated.

In [ ]:
# Print information about the folder object
print(pffp_data_folder)

## Info on the next section

The following cells walk using the prebuilt plotting functions for files and drops. For simplicity instead referencing an element of the ```pffp_data_folder.pffp_drop_files``` list (eg. ```pffp_data_folder.pffp_drop_files[0]```) the file object will be stored in a variable named ```file```. Similary, instead of referring to a drop inside of ```pffp_data_folder.pffp_drop_files[i].drops[j]``` where ```i``` can range from 0 to the number of drop files minus one and ```j``` can range from 0 to the number of drops inside that specific file, the drop object will be stored in a variable named ```drop```. 

### Exploring the pffpFile object

In [ ]:
# Store the first drop file

# The load order is different on windows and linux. For the purpose of this demonstration make sure the same file and drop is selected

# Set the indices for the file, drop indices

if os.name == "nt":
    # windows computer
    file_index, drop_index = 0, 0
elif os.name == "posix":
    # Linux computer
    file_index, drop_index = -2, 0 

file = pffp_data_folder.pffp_drop_files[file_index]

# Store the first drop
drop = pffp_data_folder.pffp_drop_files[file_index].drops[drop_index]


### Looking into the ```file```


### Printing the file meta-data

In [ ]:
print(file)

#### View the ```.bin``` file as a pandas dataframe (df)

In [ ]:
file.df

#### Manually plotting sensor information

In [ ]:
# View the units that the file.df columns are in
file.sensor_units


In [ ]:
# Store the units for the plot label
time_unit = file.sensor_units["Time"]
pressure_unit = file.sensor_units["pressure"]

# Plot the pressure data
plt.plot(file.df["Time"], file.df["pore_pressure"])
plt.xlabel(f"Time ({time_unit})")
plt.ylabel(f"Pore Pressure ({pressure_unit})")

plt.show()

#### Automatically plot the file and all the sensors

When appended to a file object ```.quick_view()``` plots the time series for each of the sensors. If ```interactive = True``` is set than the plot is interactive. Interactive plots take longer to load than non-interactive plots.

Commonly I use this plot to:

* Check that the code is identifying that the number of drops detected by the code matches what I see
* Get a general impression for what the drops look like and what the pressure response looks like
* Use the interactive plot to look at how well the sensors are calibrated. To find the start and stop of the drops the acceleration data from each of the accelerometers are "stiched" together. This means that at each data point the sensor that has the best resolution for that acceleration is used. For example if the highest acceleration from any of the sensors is measured to be 15 g's then the 18g accelerometer is trusted for that data. However, there are occasions when the 18g sensor might not be calibrarted correctly. Ploting the file data can help you see that.

**Note**: If a sensor isn't calibrated correctly the current solution is to remove it from the stitching process by not including it in the ```pffpfolder``` creating statement. For example if the 200g accelerometer isn't calibrated correctly instead of creating the folder using:

```python
pffp_data_folder.analyze_all_files(store_df=True, use_pore_pressure=True, 
                              select_accel=["2g_accel", "18g_accel", "50g_accel", "200g_accel", "250g_accel"],
                              debug = False)
```

Use:

```python

pffp_data_folder.analyze_all_files(store_df=True, use_pore_pressure=True, 
                              select_accel=["2g_accel", "18g_accel", "50g_accel", "250g_accel"],
                              debug = False)

```
where ```"200g_accel"``` has been removed from the ```selected_accel``` list

In [ ]:
# Plot the whole file
file.quick_view(interactive=True, legend=True, figsize=(8, 6))

### Exploring the ```Drop``` object

```Drop``` objects contain direct information about a single drop in a file.

Note:
* Acceleration data in the drop object includes a 1g offset down. This is so the integration of the kinematics can be done correctly.

#### Print the drop meta-data

In [ ]:
print(drop)

#### View the ```impulse_df```

The ```impulse_df``` contains the kinematics of the drop during the impact.

In [ ]:
drop.impulse_df

#### View the ```release_df```

The ```release_df``` contains the kinematics during the entire drop (ie. from release until the end of impact)

In [ ]:
drop.release_df

#### Plotting the kinematics of the entire drop

This plot goes from the point of release to the point the drop ends.

In [ ]:
drop.quick_view_release(interactive=False, legend=False, figsize = (8, 6))

#### Plotting the kinematics of the impact (impulse)

Plots the drop kinematics just during the impact.

In [ ]:
drop.quick_view_impulse(interactive=False, legend = False, figsize = (8, 6))

#### Plot the impulse selection

This plots a comparison of the part of the drop identified as the impact (impulse) and some of the drop on either end.

In [ ]:
drop.quick_view_impulse_selection(offset = 10000, legend = True, draw_line = True, line_val = 0)

In [ ]:
print(drop)

#### Set the pffp config for the drop

As the pffp config can change drop to drop. The settings need to be set for each drop individually. This could be done as a loop over all the drops if they have have the same configuration.

In [ ]:
pffp_id = pffp_data_folder.pffp_id
drop.get_pffp_tip_values(pffp_id=pffp_id, tip_type="cone", date_string = "April_2024", file_dir = tip_dir)

drop.water_drop = 1


In [ ]:

# Convert the tip values from the ones in the excel sheet to the ones used in the analysis
drop.convert_tip_vals()

#### Print the information about the tip configuration for the drop

In [ ]:
# Print the cone type
print(f"Cone type: {drop.pffp_config["tip_type"]}")

# Display the tip props df
display(drop.pffp_config["tip_props"])

### Calculate the bearing capacity

The next section is on calculating the bearing capacity. The method demonstrated here is using the historic quasi-static bearing capacity method. The White et al. method is also implemented but that isn't demonstrated here as the current soil is likely a clayey soil.

In [ ]:
# Set k_factor values that are wanted
k_factor_vals = [0.2, 0.5, 0.9, 1.2, 1.5]

# Set the drag coefficient
drag_coeff = 0.0 # 0.13830 # testing a drag coefficient of 0.13830 
for name in ["mantle", "projected"]:
    drop.calc_drop_contact_area(area_type = name)
    drop.calc_drop_dynamic_bearing(area_type = name, drag_coeff = drag_coeff)
    for val in k_factor_vals:
        drop.calc_drop_qs_bearing(k_factor = val, area_type = name)
        drop.calc_drop_qs_bearing(k_factor = val, area_type = name)



#### Showing the bearing capacity dfs

When the bearing capacity is calculated the results are stored in a dataframe. The dfs are stored as ```projected``` and ```mantle```

In [ ]:
drop.bearing_dfs.keys()

In [ ]:
drop.bearing_dfs["projected"]

In [ ]:
drop.bearing_dfs["mantle"]

#### Plot the bearing capacity

The following plots the calculated bearing capacity graphs.

In [ ]:
k_factor_vals = [0.2, 0.5, 0.9, 1.2, 1.5]

displacement = drop.impulse_df["displacement"]
df = drop.bearing_dfs["mantle"]

start = 30
# start = 0
end = -1
for val in k_factor_vals:
    column = "qsbc_mant_{}".format(val)
    k_name=  "k = {}".format(val)
    bearing = df[column][start:end]
    bearing = bearing/1e3
    plt.plot(bearing, displacement[start:end] * 100, label= k_name)

# Format the plot
plt.title("Predicted (Mantle) Bearing Capacity vs. Depth")
plt.ylabel("Penetration Depth (cm)")
plt.xlabel("QSBC (kPa)")
plt.gca().invert_yaxis()
plt.legend()

# Save the figure
save = False

if save:
    plt.savefig("Bearing_Plot.png", dpi = 300)

# Show the figure
plt.show()

#### Generate the traditional plot

There's a plan to make this into a function that lets you save the figure. I haven't gotten around to it yet.

In [ ]:
# Store  the data to make it easier to write in the plot function
df = drop.impulse_df
time = df["Time"]
accel = df["accel"]/9.81
displacement =  df["displacement"] * 100
velocity = df["velocity"]

# Create the subplot
fig, axs = plt.subplots(nrows = 1, ncols = 1, figsize = (4,5))

# Make axs into an array to make indexing easier
axs = np.atleast_1d(axs)

# Plot the data
axs[0].plot(accel, displacement, label = "Acceleration")
axs[0].plot(velocity, displacement, label = "Velocity")

# Format the plot
axs[0].set_xlabel("Acceleration (g)/Velocity (m/s)")
axs[0].set_ylabel("Penetration Depth (cm)")
axs[0].invert_yaxis()
axs[0].legend()

plt.tight_layout()

# Save the figure 
save = False
if save: 
    axs[0].savefig("accel_depth profile")

plt.show()

#### Generating the traditional plot at the file level

There's also a function at the file level that can generate the acceleration/velocity plot for all of the drops in the file.

**Note:** this function is going to be changed in the future so the plotting happens at the drop level and then this function just calls the drop function.

In [ ]:
file.plot_drop_impulses(figsize = [4,6], save_figs = False, hold = False, legend = True,
                        colors = ["black", "blue", "green", "orange", "purple", "brown"],
                        units = {"Time":"s", "accel":"g", "velocity":"m/s", "displacement":"cm"},
                        line_style = ["solid", "dashed"])